# Licenses

In [1]:
import os
import subprocess
from io import StringIO
import pandas as pd
from pathlib import Path
import urllib.request
import json

pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

def licenseFromVendor(package):
    lic = Path('./vendor/{}/LICENSE'.format(package))
    kind = 'No license found!'
    if not lic.exists():
        lic = Path('./vendor/{}/LICENSE.md'.format(package))
    if not lic.exists():
        lic = Path('./vendor/{}/LICENSE.txt'.format(package))
    if lic.exists():
        lictxt = open(lic).read().splitlines()
        kind = lictxt[0]
        if kind == '':
            kind = lictxt[1]
    return kind

def licenseFromRepo(package):
    ownerRepo = package.split('/')
    req = urllib.request.Request('https://api.github.com/repos/{}/{}/license'.format(ownerRepo[1], ownerRepo[2]))
    r = urllib.request.urlopen(req)
    j = json.loads(r.read().decode('utf-8'))
    return j['license']['key']

def maxRequests():
    req = urllib.request.Request('https://api.github.com/rate_limit')
    r = urllib.request.urlopen(req)
    j = json.loads(r.read().decode('utf-8'))
    return j

def cloneRepo(repo, owner='nats-io'):
    os.chdir('/tmp')
    repo = Path('{}'.format(repo))
    if not repo.exists():
        os.system('git clone git@github.com:/{}/{}'.format(owner, repo))
    os.chdir('/tmp/{}'.format(repo))
    
def npmLicenses(repo, owner='nats-io'):
    cloneRepo(repo, owner)
    os.system('npm install')
    stream = os.popen('license-checker --production --csv')
    out = stream.read()
    return pd.read_csv(StringIO(out))

def npmLicensesDev(repo, owner='nats-io'):
    cloneRepo(repo, owner)
    os.system('npm install')
    stream = os.popen('license-checker --direct=1 --development --csv')
    out = stream.read()
    return pd.read_csv(StringIO(out))

def goLicenses(repo, owner='nats-io'):
    cloneRepo(repo, owner)
    os.system('go mod vendor')
    f = open('go.mod')
    d = f.read()
    packages = []
    collect = False
    for line in d.splitlines():
        line = line.strip()
        if line == ')':
            collect = False
        if line == 'require (':
            collect = True
            continue
        if line.startswith('require '):
            packages.append(line.split()[1])
            continue
        if collect:
            if '// indirect' not in line:
                packages.append(line.split()[0])
            continue

    licenses = []
    for p in packages:
        kind = licenseFromVendor(p)
        if 'Copyright (c)' in kind:
            try:
                kind = licenseFromRepo(p)
            except:
                kind = licenseFromVendor(p)
        licenses.append(kind)

    d = {'package': packages, 'license': licenses}
    return pd.DataFrame(d)

## NSC

In [2]:
goLicenses('nsc')

,package,license
0,github.com/blang/semver,The MIT License
1,github.com/briandowns/spinner,Apache License
2,github.com/dustin/go-humanize,Copyright (c) 2005-2008 Dustin Sallings <dustin@spy.net>
3,github.com/mitchellh/go-homedir,The MIT License (MIT)
4,github.com/nats-io/cliprompts/v2,Apache License
5,github.com/nats-io/jwt,Apache License
6,github.com/nats-io/nats-server/v2,Apache License
7,github.com/nats-io/nats.go,Apache License
8,github.com/nats-io/nkeys,Apache License
9,github.com/nats-io/nuid,Apache License


# JWT

In [3]:
goLicenses('jwt')

,package,license
0,github.com/nats-io/nkeys,Apache License


## nats-account-server

In [4]:
goLicenses('nats-account-server')

,package,license
0,github.com/fsnotify/fsnotify,Copyright (c) 2012 The Go Authors. All rights reserved.
1,github.com/julienschmidt/httprouter,Copyright (c) 2013 Julien Schmidt. All rights reserved.
2,github.com/mitchellh/go-homedir,The MIT License (MIT)
3,github.com/nats-io/jwt,Apache License
4,github.com/nats-io/nats-server/v2,Apache License
5,github.com/nats-io/nats.go,Apache License
6,github.com/nats-io/nkeys,Apache License
7,github.com/nats-io/nsc,Apache License
8,github.com/rs/cors,Copyright (c) 2014 Olivier Poitrey <rs@dailymotion.com>
9,github.com/stretchr/testify,MIT License


## ngs

In [5]:
goLicenses('ngs', 'ConnectEverything')

,package,license
0,github.com/blang/semver,The MIT License
1,github.com/briandowns/spinner,Apache License
2,github.com/dustin/go-humanize,Copyright (c) 2005-2008 Dustin Sallings <dustin@spy.net>
3,github.com/julienschmidt/httprouter,Copyright (c) 2013 Julien Schmidt. All rights reserved.
4,github.com/lib/pq,"Copyright (c) 2011-2013, 'pq' Contributors"
5,github.com/minio/minio-go/v6,Apache License
6,github.com/mitchellh/go-homedir,The MIT License (MIT)
7,github.com/nats-io/cliprompts,Apache License
8,github.com/nats-io/jwt,Apache License
9,github.com/nats-io/nats-server/v2,Apache License


# cliprompts

In [6]:
goLicenses('cliprompts')

,package,license
0,github.com/AlecAivazis/survey,MIT License
1,github.com/mitchellh/go-homedir,The MIT License (MIT)
2,github.com/mitchellh/go-wordwrap,The MIT License (MIT)
3,github.com/stretchr/testify,Copyright (c) 2012 - 2013 Mat Ryer and Tyler Bunnell


## nats.js

In [7]:
npmLicenses('nats.js')

,module name,license,repository
0,nats@1.4.9,Apache-2.0,https://github.com/nats-io/nats.js
1,nuid@1.1.4,Apache-2.0,https://github.com/nats-io/node-nuid
2,ts-nkeys@1.0.16,Apache-2.0,https://github.com//nats-io/ts-nkeys
3,tweetnacl@1.0.3,Unlicense,https://github.com/dchest/tweetnacl-js


## node-nuid

In [8]:
npmLicenses('node-nuid')

,module name,license,repository
0,nuid@1.1.4,Apache-2.0,https://github.com/nats-io/node-nuid


## ts-nkeys

In [9]:
npmLicenses('ts-nkeys')

,module name,license,repository
0,ts-nkeys@1.0.16,Apache-2.0,https://github.com//nats-io/ts-nkeys
1,tweetnacl@1.0.3,Unlicense,https://github.com/dchest/tweetnacl-js


## stan.js

In [10]:
npmLicenses('stan.js')

,module name,license,repository
0,google-protobuf@3.11.2,BSD-3-Clause,https://github.com/protocolbuffers/protobuf/tree/master/js
1,nats@1.4.6,Apache-2.0,https://github.com/nats-io/nats.js
2,node-nats-streaming@0.3.0,Apache-2.0,https://github.com/nats-io/stan.js
3,nuid@1.1.4,Apache-2.0,https://github.com/nats-io/node-nuid
4,ts-nkeys@1.0.16,Apache-2.0,https://github.com//nats-io/ts-nkeys
5,tweetnacl@1.0.3,Unlicense,https://github.com/dchest/tweetnacl-js


## nats.ts

In [11]:
npmLicenses('nats.ts')

,module name,license,repository
0,nuid@1.1.4,Apache-2.0,https://github.com/nats-io/node-nuid
1,ts-nats@1.2.14,Apache-2.0,https://github.com/nats-io/nats.ts
2,ts-nkeys@1.0.16,Apache-2.0,https://github.com//nats-io/ts-nkeys
3,tweetnacl@1.0.3,Unlicense,https://github.com/dchest/tweetnacl-js


## nats.ws

In [12]:
npmLicenses('nats.ws')

,module name,license,repository
0,js-nuid@1.1.1-4,Apache-2.0,https://github.com//nats-io/js-nuid
1,nats.ws@1.0.0-17,Apache-2.0,https://github.com/nats-io/nats.ws


## js-nuid

In [13]:
npmLicenses('js-nuid')

,module name,license,repository
0,js-nuid@1.1.1-4,Apache-2.0,https://github.com//nats-io/js-nuid


## node-nats-examples - this is examples for docs.nats.io

In [14]:
npmLicenses('node-nats-examples')

,module name,license,repository
0,node-nats-examples@1.0.0,Apache-2.0,https://github.com//nats-io/node-nats-examples


## ts-nats-examples - this is examples for docs.nats.io

In [15]:
npmLicenses('ts-nats-examples')

,module name,license,repository
0,nuid@1.1.4,Apache-2.0,https://github.com/nats-io/node-nuid
1,ts-nats-examples@1.0.0,Apache-2.0,https://github.com//nats-io/ts-nats-examples
2,ts-nats@1.2.12,Apache-2.0,https://github.com/nats-io/nats.ts
3,ts-nkeys@1.0.16,Apache-2.0,https://github.com//nats-io/ts-nkeys
4,tweetnacl@1.0.3,Unlicense,https://github.com/dchest/tweetnacl-js
